In [1]:
import os, random, time
import numpy as np, numpy.linalg as la, scipy as sp, matplotlib.pyplot as plt, matplotlib.ticker as ticker, matplotlib.animation as animation, scipy.constants as ct, scipy.optimize as sciopt
from IPython.display import HTML
from IPython.display import set_matplotlib_formats
%matplotlib tk
plt.rcParams.update({'font.size': 20})
set_matplotlib_formats('pdf', 'svg')
from collections import defaultdict
from math import *
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d, InterpolatedUnivariateSpline, RegularGridInterpolator, RectBivariateSpline
from scipy.signal import savgol_filter, find_peaks, hann, hilbert
from scipy.optimize import minimize, curve_fit
from scipy.misc import derivative
from numpy.fft import rfft, rfftfreq, irfft

import PyLTSpice
from PyLTSpice.LTSpice_RawRead import LTSpiceRawRead
from PyLTSpice.LTSpiceBatch import LTCommander

import VoltageGeneration

Found Numpy. WIll be used for storing data


# Initialize Trap

In [4]:
#### MaxBeta
reload_vg = True
amu = ct.m_u
e = ct.e
Ca = 40*amu
Sr = 88*amu

electrode_ordering = [
    'S21', 'N1', 'S20', 'N2', 'S19', 'N3', 'S14',
    'N4', 'S13', 'N8', 'S12', 'N9', 'S11', 'N10',
    'S10', 'N11', 'S9', 'N12', 'S8', 'N13', 'S4',
    'N14', 'S3', 'N19', 'S2', 'N20', 'S1', 'N21',
    'S25', 'S24', 'S23', 'S22'
]

path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/GridFiles'
name = 'MaxBeta_Production_Potential'

electrode_grouping = [[i+1] for i in range(len(electrode_ordering))]
# electrode_grouping = [[i+1] for i in range(len(electrode_ordering)) if i not in [18,19,22,23]]
# electrode_grouping.extend([[19,20],[23,24]])
print(electrode_grouping)

d_cons = []
custom_fit = ['x','y','z','xx','xy','xz','yy','yz','zz','yyy','yyyy','yyyyy','yyyyyy']

start_time = time.time()
if reload_vg:
    vg = VoltageGeneration.VoltageGeneration(path, name, electrode_grouping=electrode_grouping, fit_ranges=[5e-6, 10e-6, 5e-6], rf_electrodes=[33], m=Ca, rf_omega=2.*np.pi*45.937e6, order=4, f_cons=d_cons, v_rf=[72])
print("Imported grid files in {}".format(time.time() - start_time))


[[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]
Importing electrode 9
Importing electrode 8
Importing electrode 23
Importing electrode 22
Importing electrode 20
Importing electrode 21
Importing electrode 19
Importing electrode 31
Importing electrode 25
Importing electrode 24
Importing electrode 30
Importing electrode 18
Importing electrode 26
Importing electrode 32
Importing electrode 33
Importing electrode 27
Importing electrode 16
Importing electrode 17
Importing electrode 15
Importing electrode 29
Importing electrode 28
Importing electrode 14
Importing electrode 10
Importing electrode 11
Importing electrode 13
Importing electrode 12
Importing electrode 1
Importing electrode 3
Importing electrode 2
Importing electrode 6
Importing electrode 7
Importing electrode 5
Importing electrode 4
Potentials imported. (dev version)
Imported grid files in 107.0493

In [ ]:
#### SLT
reload_vg = False
amu = ct.m_u
e = ct.e
Ca = 40*amu
Sr = 88*amu

electrode_ordering = [
    "S6", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
    "S24", "S25",
    "N6", "N7", "N8", "N9", "N10", "N11", "N12", "N13", "N14",
] ## Ordering for the SLT configuration

path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/GridFiles'
name = 'AngleTrap_Skinny_Potential_reduced'

electrode_grouping = [[i+1] for i in range(len(electrode_ordering))]
d_cons = []

start_time = time.time()
if reload_vg:
    vg = VoltageGeneration.VoltageGeneration(path, name, electrode_grouping=electrode_grouping, fit_ranges=[5e-6, 10e-6, 5e-6], rf_electrodes=[21], m=Ca, rf_omega=2.*np.pi*45.247e6, order=4, f_cons=d_cons, v_rf=[50])
print("Imported grid files in {}".format(time.time() - start_time))


# Set Ideal Motion Protocol

In [5]:
def qtanhN(t, tspan, N=5, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T = tf-t0
    if (t-t0) < T:
        return y0 + max((yf-y0)/2*(np.tanh(N*(2*(t-t0)-T)/T)+np.tanh(N))/np.tanh(N), 0)
    else:
        return yf
    
def qsin(t, tspan, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T = tf-t0
    if t < t0:
        return y0
    elif (t-t0) < T:
        return y0 + max((yf-y0)/2*(1-np.cos(pi*(t-t0)/T)), 0)
    else:
        return yf

def qsta(t, tspan, qspan=(0, 120), f=1):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*((1/(2*pi*f)**2)*(60*s - 180*s**2 + 120*s**3) + 10*s**3 - 15*s**4 + 6*s**5)
    else:
        return yf
    
def qsta2(t, tspan, qspan=(0, 120), f=1):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*((1/(2*pi*f)**2)*(2520*s**3-12600*s**4+22680*s**5-17640*s**6+5040*s**7)
                             + 126*s**5-420*s**6+540*s**7-315*s**8+70*s**9)
    else:
        return yf
    
def qtrig(t, tspan, qspan=(0, 120), f=1.2e6, mu=1):
    ''' args should be f (axialfreq in Hz), mu (mass ratio)'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti, d = tf-t0, t-t0, yf-y0
    s = ti/T
    Op, On = 2*np.pi*f*(1+1/mu+(1-1/mu+1/mu**2)**0.5)**0.5, 2*np.pi*f*(1+1/mu-(1-1/mu+1/mu**2)**0.5)**0.5
    b3 = -49*((T*Op)**2 - (5*np.pi)**2)*((T*On)**2 - (5*np.pi)**2)/(2048*(T*T*On*Op)**2)
    b4 = 5*((T*Op)**2 - (7*np.pi)**2)*((T*On)**2 - (7*np.pi)**2)/(2048*(T*T*On*Op)**2)
    
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + d*(0.5 + (-9/16+2*b3+5*b4)*np.cos(np.pi*s) + 1/16*(1-48*b3-96*b4)*np.cos(3*np.pi*s) + b3*np.cos(5*np.pi*s) + b4*np.cos(7*np.pi*s))
    else:
        return yf
    
def d_t(t, tspan, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*(10*s**3 - 15*s**4 + 6*s**5)
    else:
        return yf

def get_freq_from_alpha(a, m_=40*ct.u):
    ''' make sure alpha is in V/um^2'''
    return np.sign(a)*(2*np.abs(a)*1e12*ct.e/m_)**0.5/(2*pi)

def get_alpha_from_freq(f, m_=40*ct.u):
    ''' returns alpha with units of V/m^2'''
    return (2*pi*f)**2/(2*ct.e/m_)

def p_derivative(func, ms=1e-11, var=1, point=[], order=1):
    ''' implements a partial derivative '''
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return derivative(wraps, point[var], dx = ms, n=order)

def gaussian(x, mu, sig):
    return np.exp(-(x-mu)*(x-mu)/(2*sig*sig))

## Transport

In [6]:
f0 = 4e6 # Axial frequency [Hz]
T0 = 1.96e-6 # Transport time [s]
move = (186.5, 231.5) # Start and End positions [um]
a0 = get_alpha_from_freq(f0, m_=Ca)*1e-12 # Specify ion mass. [Units are V/m^2, but ITVG uses um as the length scale]

t_ideal = np.linspace(0, T0, 100)
q_ideal = np.array([qsta2(t, (0, T0), qspan=move, f=T0*f0) for t in t_ideal])
# q_ideal = np.linspace(move[0], move[1], len(t_ideal))
a_ideal = np.repeat(a0, len(t_ideal))
q_ideal_func = interp1d(np.append(t_ideal, 1), np.append(q_ideal, q_ideal[-1]))
a_ideal_func = interp1d(np.append(t_ideal, 1), np.append(a_ideal, a_ideal[-1]))

plt.plot(t_ideal*1e6, q_ideal)
plt.xlabel('time (us)')
plt.ylabel('position (um)')

Text(0, 0.5, 'position (um)')

In [ ]:
f0 = 0.4e6 # Axial frequency [Hz]
a0 = get_alpha_from_freq(f0, m_=Ca)*1e-12 # Specify ion mass. [Units are V/m^2, but ITVG uses um as the length scale]

T0 = 10e-6
t_ideal = np.array(np.linspace(0, T0, 10))
q_ideal = np.array([-186.5]*10)
# a_ideal = np.repeat(a0, len(t_ideal))
a_ideal = [get_alpha_from_freq(f, m_=Ca)*1e-12 for f in np.linspace(0.4e6, 4e6, 10)]


In [ ]:
#### bang bang (bang)
f0 = 2.5e6
a0 = get_alpha_from_freq(f0, m_=Ca)*1e-12
dt = 20e-6

t_ideal = np.array([0, 1, 2, 3])*dt
q_ideal = np.array([0, 40, 80, 120])
a_ideal = np.repeat(a0, len(t_ideal))
T0 = t_ideal[-1]

plt.plot(t_ideal*1e6, q_ideal, marker='.')

# Find Voltages
**Aka, backwards ITVG**

## Transport

In [10]:
def rotate_coeffs(theta, xx, xz, zz):
    """
    Find the coefficients $C'_{xx}, C'_{xz}, C'_{zz}$ of the potential
    after a rotation in the zx-plane by `theta` given the unprimed values,
    as described above.
    
    `theta` specified in radians.
    """
    sin = np.sin
    cos = np.cos
    
    xxnew = xx*cos(theta)**2 + zz*sin(theta)**2 + 0.5*xz*sin(2*theta)
    xznew = xz*cos(2*theta)  + (zz-xx)*sin(2*theta)
    zznew = zz*cos(theta)**2 + xx*sin(theta)**2 - 0.5*xz*sin(2*theta)
    
    return [xxnew, xznew, zznew]

def dcTiltAngle(vg, R, theta_targ, yy, rfEls, delta=3, debug=False):
    # expansion coefficients of the pseudopotential near the trapping point `R`
    # THIS FUNCTION ASSUMES ONE RF ELECTRODE!
    rf_coeffs = vg.printCoefficients(R, rfEls, ('zz', 'xx', 'xz', 'yy', 'xy', 'yz'), printing=False)[0]
    [rfzz, rfxx, rfxz, rfyy, rfxy, rfyz] = rf_coeffs[:,0]
    
    def tiltError(theta_tilt, debug=False):
        """
        Error in the tilt angle of the full potential, as a function of the dc tilt.
        """
        
        zz = delta*yy
        xx = -(1+delta)*yy # delta ensures Laplace (div phi = 0) is satisfied, but adjusts the principal axes of the xz ellipse
        xz = 0
        
        # rotate!
        [xxnew, xznew, zznew] = rotate_coeffs(theta_tilt, xx, xz, zz)

        if debug:
            print (np.degrees(theta_tilt))
            print (np.degrees(np.arctan((xznew+rfxz)/((rfzz+zznew) - (rfxx+xxnew))))/2)
        
        return (
            np.tan(2*theta_targ) - (xznew+rfxz)/((rfzz+zznew) - (rfxx+xxnew))
        )**2
    
    if debug:
        pass
#         print (np.degrees(theta_targ))

#         fig, ax = plt.subplots()
#         angle_space = np.linspace(0, np.pi/2, 91)
#         errors = [tiltError(theta) for theta in angle_space]
#         ax.plot(np.degrees(angle_space), errors)
#         ax.set_ylim(0,1)
    
    return sciopt.minimize_scalar(tiltError).x

def tiltWithPseudo(vg, R, theta_targ, yy, rfEls, delta=3, debug=False):
    th = dcTiltAngle(vg, R, theta_targ, yy, rfEls, delta=delta, debug=debug)
    
    if debug:
        print ("Tilting dc to {:.2f} deg".format(np.degrees(th)))
    
    zz = delta*yy
    xx = -(1+delta)*yy
    xz = 0
    return rotate_coeffs(th, xx, xz, zz)


In [8]:
MB_intervals = {14:[0,74], 13:[74,119], 12:[119,164], 11:[164,209], 10:[209,254], 9:[254,299], 8:[299,373],
               7:[373,447], 6:[447,492], 5:[492,537], 4:[537,582], 3:[582,627], 2:[627,672], 1:[672,746],
               15:[-74,0], 16:[-119,-74], 17:[-164,-119], 18:[-209,-164], 19:[-254,-209,], 20:[-299,-254], 21:[-373,-299],
               22:[-447,-373], 23:[-492,-447], 24:[-537,-492], 25:[-582,-537], 26:[-627,-582], 27:[-672,-627], 28:[-746,-672]} # MaxBeta electrode intervals

SLT_intervals = {1:[], 2:[-420,-300], 3:[-300,-180], 4:[-180,-60], 5:[-60,60], 6:[60,180], 7:[180,300], 8:[300,420], 9:[], 12:[], 20:[],
                19:[-420,-300], 18:[-300,-180], 17:[-180,-60], 16:[-60,60], 15:[60,180], 14:[180,300], 13:[300,420]}

def getOverlap(a, b):
    return max(0, min(a[1], b[1]) - max(a[0], b[0]))

def get_constrained_voltages(q, tilt_el={'S25': 12.930878042522904, 'S24': -24.036409224366135, 'S23': -39.5, 'S22': 27.90731149920946}):
    tilt_el_num = [29,30,31,32]

    if q<=-604.5:
        volt = [(p[0]-1, qsta2(q, (74, 119), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[25,26,27,28]]
    elif q<=-559.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[24,25,26,27]]
    elif q<=-514.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[23,24,25,26]]
    elif q<=-469.5:
        volt = [(p[0]-1, qsta2(q, (254, 299), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[22,23,24,25]]
    elif q<=-410:
        volt = [(p[0]-1, qsta2(q, (254, 299), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[21,22,23,24]]
    elif q<=-336:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[20,21,22,23]]
    elif q<=-276.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[19,20,21,22]]
    elif q<=-231.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[18,19,20,21]]
    elif q<=-186.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[17,18,19,20]]
    elif q<=-141.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[16,17,18,19]]
    elif q<=-86:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[15,16,17,18]]
    elif q<=-21.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[14,15,16,17]]  
    elif q<=21.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[13,14,15,16]]
    elif q<=86:
        volt = [(p[0]-1, qsta2(q, (74, 119), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[12,13,14,15]]
    elif q<=141.5:
        volt = [(p[0]-1, qsta2(q, (74, 119), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[11,12,13,14]]
    elif q<=186.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[10,11,12,13]]
    elif q<=231.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[9,10,11,12]]
    elif q<=276.5:
        volt = [(p[0]-1, qsta2(q, (254, 299), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[8,9,10,11]]
    elif q<=336:
        volt = [(p[0]-1, qsta2(q, (254, 299), (8, 8), f=2)) for p in electrode_grouping if p[0] not in tilt_el_num+[7,8,9,10]]
    elif q<=410:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[6,7,8,9]]
    elif q<=469.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[5,6,7,8]]
    elif q<=514.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[4,5,6,7]]
    elif q<=559.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[3,4,5,6]]
    elif q<=604.5:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[2,3,4,5]]
    else:
        volt = [(p[0]-1, 8) for p in electrode_grouping if p[0] not in tilt_el_num+[1,2,3,4]]
        
    volt += [(p-1, tilt_el[electrode_ordering[p-1]]) for p in tilt_el_num]
    return volt


In [11]:
tilt_angle = np.radians(10) # specify tilt [rad]
volt_gen = vg
bounds = (-39.5, 39.5)
norm = np.full((1, 1), 1) # normalization factor to pull out certain coefficients
# norm[[1,4]] = 5e1 # y and yy coefficients
# norm[[7,8]] = 1e2
volt_gen.set_electrode_grouping(electrode_grouping) # sets previous voltages to None

allvoltages = []
for q, a, i in zip(q_ideal, a_ideal, 1+np.arange(len(t_ideal))):
    print(i, 'solving at (q, a):', q, a)
    R = [[0, q, 50.0]] # trap center
    constrained_voltages = get_constrained_voltages(q, tilt_el={'S25':0, 'S24':0, 'S23':0, 'S22':0})
    print('constrained voltages: ', constrained_voltages)
    xx, xz, zz = tiltWithPseudo(volt_gen, R, tilt_angle, a, volt_gen.rf_electrodes, debug=False)
    cons = [('x', 0), ('y', 0), ('z', 0), ('yy', a), ('xy', 0), ('yz', 0)]
    nom_v = volt_gen.findControlVoltages(R, cons=[cons], tol=1e-27, fixed_voltages=constrained_voltages, bnds=bounds, normvec=norm, epss=1e-12, independent=True)
    allvoltages.append({electrode_ordering[num-1]: v for [num, v] in volt_gen.ungroup_configuration(nom_v)})
    

1 solving at (q, a): 186.5 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '2.1959410688542928e-08']
 ['y' '0' '-3.18801002307037e-12']
 ['z' '0' '1.4590826376975619e-12']
 ['xy' '0' '1.8878186295510322e-08']
 ['yy' '0.00013093278896406064' '0.00013093278074398135']
 ['yz' '0' '-7.086185479544815e-13']]
Final cost value:  4.845581847692899e-07
Number of iterations:  53 

2 solving at (q, a): 186.5000463510298 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized C

Target, Realized Coeffs:
 [['x' '0' '2.0272368280965066e-08']
 ['y' '0' '3.6412521760807726e-12']
 ['z' '0' '1.403121325724288e-12']
 ['xy' '0' '1.8743347552637146e-08']
 ['yy' '0.00013093278896406064' '0.00013093278106811698']
 ['yz' '0' '6.27112328266486e-12']]
Final cost value:  4.794214849624262e-07
Number of iterations:  51 

14 solving at (q, a): 186.6907655514147 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '2.1143455135588973e-08']
 ['y' '0' '3.708517271484163e-12']
 ['z' '0' '1.4271455111430909e-12']
 ['xy' '0' '1.8754822911312326e-08']
 ['yy' '0.00013093278896406064' '0.0001309327809575401']
 ['yz' '0' '6.409182714173407e-12']]
Final cost value:  4.806410537313037e-07
Number of iterations:  63 

15 

Target, Realized Coeffs:
 [['x' '0' '5.826062400579199e-08']
 ['y' '0' '6.44687612755962e-12']
 ['z' '0' '2.642267264360565e-12']
 ['xy' '0' '1.935271300083825e-08']
 ['yy' '0.00013093278896406064' '0.0001309327770709593']
 ['yz' '0' '1.2629413668834542e-11']]
Final cost value:  5.647454159642894e-07
Number of iterations:  66 

27 solving at (q, a): 189.33554380970682 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '6.390489107049232e-08']
 ['y' '0' '6.521928938747756e-12']
 ['z' '0' '2.8175532653523794e-12']
 ['xy' '0' '1.8741239166014685e-08']
 ['yy' '0.00013093278896406064' '0.0001309327771856618']
 ['yz' '0' '1.3089835720339268e-11']]
Final cost value:  5.671131730530445e-07
Number of iterations:  61 

28 so

Target, Realized Coeffs:
 [['x' '0' '9.874278025538461e-08']
 ['y' '0' '7.81774298075355e-12']
 ['z' '0' '3.748659369029461e-12']
 ['xy' '0' '6.798788339043607e-09']
 ['yy' '0.00013093278896406064' '0.00013093278269651892']
 ['yz' '0' '1.747039818855104e-11']]
Final cost value:  5.221525977176814e-07
Number of iterations:  63 

40 solving at (q, a): 198.06624743024076 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '9.858495735392271e-08']
 ['y' '0' '7.884043244643646e-12']
 ['z' '0' '3.800430998547599e-12']
 ['xy' '0' '8.354440695009553e-09']
 ['yy' '0.00013093278896406064' '0.00013093278320970743']
 ['yz' '0' '1.7474342732894978e-11']]
Final cost value:  5.353484691737673e-07
Number of iterations:  55 

41 sol

Target, Realized Coeffs:
 [['x' '0' '3.9277765785524046e-08']
 ['y' '0' '2.349317788918892e-12']
 ['z' '0' '1.4831522511873918e-12']
 ['xy' '0' '-1.0690177313303353e-08']
 ['yy' '0.00013093278896406064' '0.00013093279002976464']
 ['yz' '0' '4.861821923242242e-12']]
Final cost value:  3.316527094301394e-07
Number of iterations:  47 

53 solving at (q, a): 211.75043344335012 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '2.9571293015219354e-08']
 ['y' '0' '1.841852625278362e-12']
 ['z' '0' '1.0960084294124595e-12']
 ['xy' '0' '-7.953458443295815e-09']
 ['yy' '0.00013093278896406064' '0.0001309327901245688']
 ['yz' '0' '3.721603645916172e-12']]
Final cost value:  2.477851531742912e-07
Number of iterations:  52 



Target, Realized Coeffs:
 [['x' '0' '1.1310838362735206e-07']
 ['y' '0' '8.794536496617433e-12']
 ['z' '0' '6.879780599378105e-12']
 ['xy' '0' '1.7475678881972593e-08']
 ['yy' '0.00013093278896406064' '0.00013093280279781051']
 ['yz' '0' '1.5091515615851293e-11']]
Final cost value:  7.14641533555024e-07
Number of iterations:  52 

66 solving at (q, a): 224.09478481084042 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '1.2392283619220065e-07']
 ['y' '0' '9.443242195150692e-12']
 ['z' '0' '7.464167305071268e-12']
 ['xy' '0' '1.639776175712581e-08']
 ['yy' '0.00013093278896406064' '0.00013093280452120432']
 ['yz' '0' '1.60713638442311e-11']]
Final cost value:  7.429509867536763e-07
Number of iterations:  61 

67 s

Target, Realized Coeffs:
 [['x' '0' '1.7702411251611612e-07']
 ['y' '0' '1.5753659878339765e-11']
 ['z' '0' '9.45895293515131e-12']
 ['xy' '0' '6.712007287961812e-10']
 ['yy' '0.00013093278896406064' '0.00013093282179539096']
 ['yz' '0' '2.1250532705589277e-11']]
Final cost value:  8.852801498291561e-07
Number of iterations:  66 

79 solving at (q, a): 230.24690872324237 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '1.7817232330367415e-07']
 ['y' '0' '1.5811194367665315e-11']
 ['z' '0' '9.548159355179942e-12']
 ['xy' '0' '-1.94238634653596e-09']
 ['yy' '0.00013093278896406064' '0.00013093282186194444']
 ['yz' '0' '2.1262584019312268e-11']]
Final cost value:  8.921846376980549e-07
Number of iterations:  59 

8

Target, Realized Coeffs:
 [['x' '0' '1.8409255904826226e-07']
 ['y' '0' '1.658548829958778e-11']
 ['z' '0' '9.657582025757172e-12']
 ['xy' '0' '-4.4739859894915824e-09']
 ['yy' '0.00013093278896406064' '0.00013093282471047927']
 ['yz' '0' '2.1354682060745027e-11']]
Final cost value:  9.272341637480483e-07
Number of iterations:  50 

92 solving at (q, a): 231.46914771973024 0.00013093278896406064
constrained voltages:  [(0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8), (19, 8), (20, 8), (21, 8), (22, 8), (23, 8), (24, 8), (25, 8), (26, 8), (27, 8), (28, 0), (29, 0), (30, 0), (31, 0)]
Target, Realized Coeffs:
 [['x' '0' '1.8414206530776877e-07']
 ['y' '0' '1.660339617529094e-11']
 ['z' '0' '9.6763439275116e-12']
 ['xy' '0' '-4.4810076834668314e-09']
 ['yy' '0.00013093278896406064' '0.00013093282474405666']
 ['yz' '0' '2.135375483749459e-11']]
Final cost value:  9.275010746135449e-07
Number of iterations:  48 

9

# View Ideal Voltages

In [18]:
d = defaultdict(list)
for f in allvoltages:
    [d[k].append(v) for k, v in f.items()]

dt = 0.1e-6 # sample rate of DAC
N0 = int(T0/dt)
buffer1 = 1
buffer2 = 1
Ntot = buffer1+N0+buffer2

v_ideal = {} # voltages from backwards ITVG
v_DAC = {} # sampled voltages for DAC commands
plt.figure('voltages')
for k, v in d.items():
    mark = '.'
    if k[0] == 'S':
        mark = 'x'

    vevents = np.concatenate(([v[0]], v, [v[-1]]))
    tevents = np.concatenate(([-0.1e-6], dt*np.linspace(buffer1, N0+buffer1, len(v)), [Ntot*dt+0.1e-6]))

    if k not in []:
        plt.plot([t*1e6 for t in tevents], vevents, marker=mark, label=k)
    v_ideal[k] = interp1d(np.append(tevents, 1), np.append(vevents, vevents[-1]))
    v_DAC[k] = v_ideal[k].__call__([(n+0)*dt for n in range(Ntot)])
plt.legend(loc='best', prop={'size': 12})
plt.ylabel('Electrode Voltage (V)')
plt.xlabel('Time (us)')
endsim = dt*len(v_DAC['N9'])


**Save Voltages**

In [19]:
#### save allvoltages
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
# filepath = 'MB_MBZ2-MBZ3_2500kHz.npy'
filepath = 'MB_split_max.npy'

np.save(os.path.join(basepath, filepath), allvoltages)

In [ ]:
#### saves voltages as a list of dicts
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filepath = 'MB_twostagesplit_1000kHz_zerotilt_z=52um.npy'

newdat = []
for i in range(len(v_DAC['N10'])):
    newdat.append({k: v[i] for k, v in v_DAC.items() if k not in ['S22', 'S23', 'S24', 'S25']})
newdat = np.array(newdat)

# tilt_el={'S25': 12.930878042522904, 'S24': -24.036409224366135, 'S23': -39.5, 'S22': 27.90731149920946}
tilt_el={'S25': 0, 'S24': 0, 'S23': 0, 'S22': 0}

for dic in newdat:
    for k in ['S22', 'S23', 'S24', 'S25']:
        dic[k] = tilt_el[k]
print(newdat)
np.save(os.path.join(basepath, filepath), newdat) 


# Simulate Trap Effects
**Potential sources of noise include: DAC slew rate, calibration and gain offset, crosstalk, discrete inputs, filtering, segmentation of electrodes**

Important to characterize between physical and numerical effects

## Simulate Electrode Response

In [15]:
def set_vinput(volt_array, dt, st, sr):
    ''' writes PWL input to 'input.txt' '''    
    N = len(volt_array)
    varray = [v/16+2.5 for v in volt_array] ## fastino
#     varray = volt_array ## zotino
    
    tevents = [0]
    vevents = [varray[0]]
    for i in range(1,N):
        ti = dt*i
        vi = vevents[-1]
        tevents.append(ti)
        vevents.append(vi)
        
        tdelay = abs(varray[i] - vi)/sr
#         tdelay = 1e-12
        if tdelay > dt:
            print('WARNING: SLEW RATE LONGER THAN TIME STEP')
            tevents.append(ti + dt)
            vevents.append(vi + (varray[i] - vi)*dt/tdelay)
        else:
            tevents.append(ti + tdelay)
            vevents.append(varray[i])
    tevents.append(N*dt)
    vevents.append(varray[-1])
    
    with open(circuit_directory_path + 'input.txt', 'w+') as f:
        [f.write('\t'.join((str(t), str(v))) + '\n') for t, v in zip(tevents, vevents)]

    return tevents, vevents

def run_spice_sim(endsim, st, attempts=1):
    instruction = '.TRAN' + ' '.join((str(st), str(endsim)))
    LTC.netlist.insert(-2, instruction)
    LTC.write_netlist()
    rawfile, logfile = LTC.run()
    LTC.reset_netlist()

    if rawfile:
        for i in range(attempts):
            try:
                LTR = LTSpiceRawRead(rawfile)
                break
            except UnicodeDecodeError:
                LTC.netlist.insert(-2, instruction)
                LTC.write_netlist()
                rawfile, logfile = LTC.run()
                LTC.reset_netlist()
                LTR = LTSpiceRawRead(rawfile)
        os.remove(logfile)
    else:
        print('SIMULATION FAILED')
        with open(logfile, 'r') as f:
            [print(line) for line in f.readlines()]
        os.remove(logfile)
        return None
    return LTR

circuit_name = 'FastinoAmplifier_V1.0.cir' ## Zotino vs FastinoAmplifier vs minimalfiltering

circuit_directory_path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/lib_circuits/'
LTC = LTCommander(circuit_directory_path + circuit_name)
endsim = T0
st = max(dt/20, 1e-9) ## maximum time step of the SPICE transient sim. no lower than 100ns
slew_rate = 17/1e-6 ## slew rate in V/s (~17V/us)
gain_err = 0.01 ## 1%
calibration_err = 0.1 ## +- 100mV

print(dt/st, 'sim steps per dt')

20.0 sim steps per dt


- DAC discrete commands, slew rate, and filtering are naturally taken care of.
- we can artificially add calibration + gain offsets, as well as crosstalk...
- anything else??? John: johnson noise? electric field noise?

In [22]:
plotting = False
addnoise = False
endbuffer = 30e-6/dt

v_exp = {}
for k, v in v_DAC.items():
    if k not in []:
        endsim = dt*(len(v) + endbuffer)
        tevents, vevents = set_vinput(v[::], dt, st, slew_rate)

        LTR = run_spice_sim(endsim, st, 3)

        analysis_output = LTR.get_trace('V(ion)').get_wave(0)
        t_spice = LTR.get_trace('time').get_time_axis(0)
        print('\t'.join((k, str(v), str(t_spice[-1]))), '\n')

        if addnoise:
            v_exp[k] = interp1d(t_spice, np.array(analysis_output)*(1+random.choice([-gain_err, gain_err]))
                             + random.choice([-calibration_err, calibration_err]))
        else:
            v_exp[k] = interp1d(np.concatenate([t_spice, [1]]), np.concatenate([analysis_output, [analysis_output[-1]]]))

        if plotting:
            plt.figure(k, (19, 9))
            plt.title('Electrode %s' % (k, ))
            plt.xlabel('Time [us]')
            plt.ylabel('Voltage [V]')
            plt.grid()

            t_view = np.linspace(0, t_spice[-1], 1000)
            plt.plot([t*1e6 for t in tevents], [16*(v-2.5) for v in vevents], label='input') ## fastino
    #             plt.plot([t*1e6 for t in tevents], vevents, label='input') ## zotino
            plt.plot(t_spice*1e6, analysis_output, label='output')
            plt.plot(t_view*1e6, v_ideal[k].__call__(t_view), 'k--', label='ideal')
            plt.legend(loc='best')

            plt.tight_layout()
plt.show()

Tue Jul 27 11:27:26 2021 : Starting simulation 9
Tue Jul 27 11:27:27 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
S21	[8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]	3.21e-05 

Tue Jul 27 11:27:27 2021 : Starting simulation 10
Tue Jul 27 11:27:28 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
N1	[8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]	3.21e-05 

Tue Jul 27 11:27:28 2021 : Starting simulation 11
Tue Jul 27 11:27:28 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
S20	[8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]	3.21e-05 

Tue Jul 27 11:27:28 2021 : Starting simulation 12
Tue Jul 27 11:27:29 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
N2	[8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]	3.21e-05 

Tue Jul 27 11:27:29 2021 : Starting simulation 13
Tue Jul 27 11:27:30 2021: Simulation Successful. Time elapsed 00:00:00:

Normal acces

## Calculate Potential

In [23]:
endITVG = endsim - 0e-6 # we can't find the potential for voltages that don't exist! hence the minus 0.1us
volt_gen = vg
N_final = int(endITVG/100e-9)+1 # set dt (time resolution)
N_final = len(d['N9'])
t_exp, dx = np.linspace(0, (endITVG), N_final, retstep=True)
print(len(t_exp), 'calls per iteration \t', endITVG, 'us')

## DON'T FORGET TO CHOOSE PROPER VOLTAGES

bigarray = np.array([v_exp[k].__call__(t_exp) for k in electrode_ordering]).T
# bigarray = np.array([v_DAC[k] for k in electrode_ordering]).T
# bigarray = np.array([d[k] for k in electrode_ordering]).T
el_config = []
for row in bigarray:
    el_configi = [[[i+1], v] for i, v in enumerate(row)]
    el_configi.extend([[[el], 1] for el in volt_gen.rf_electrodes])
    el_config.append(el_configi)
    

100 calls per iteration 	 3.21e-05 us


In [24]:
el_config

[[[[1], 8.048927307128906],
  [[2], 8.048927307128906],
  [[3], 8.048927307128906],
  [[4], 8.048927307128906],
  [[5], 8.048927307128906],
  [[6], 8.048927307128906],
  [[7], 8.048927307128906],
  [[8], 8.048927307128906],
  [[9], 8.048927307128906],
  [[10], 1.0989230871200562],
  [[11], 0.6320891976356506],
  [[12], 1.1083329916000366],
  [[13], 7.974058151245117],
  [[14], 8.048927307128906],
  [[15], 8.048927307128906],
  [[16], 8.048927307128906],
  [[17], 8.048927307128906],
  [[18], 8.048927307128906],
  [[19], 8.048927307128906],
  [[20], 8.048927307128906],
  [[21], 8.048927307128906],
  [[22], 8.048927307128906],
  [[23], 8.048927307128906],
  [[24], 8.048927307128906],
  [[25], 8.048927307128906],
  [[26], 8.048927307128906],
  [[27], 8.048927307128906],
  [[28], 8.048927307128906],
  [[29], 0.0004523532697930932],
  [[30], 0.0004523532697930932],
  [[31], 0.0004523532697930932],
  [[32], 0.0004523532697930932],
  [[33], 1]],
 [[[1], 8.048927307128906],
  [[2], 8.0489273071

In [32]:
down = 1
R = [[0.0, 0, 51]]
y_span_um = (100, 370)
# y_span_um = (0, 373)
n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9)+1 # set dy (space resolution)

y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
print(y_points)
y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config[::down]])
xaxis = y_points
yaxis = y_pot
adaptive_yaxis = True
q_min = [y_points[np.where(np.array(y) == min(np.array(y)))][0] for y in y_pot]

# fig, ax = plt.subplots()
# # ax.set_ylim(min(yaxis.flatten()), max(yaxis.flatten()))
# ax.set_ylim(-2, 2)
# line, = ax.plot(xaxis, yaxis[0])

# def animate(i):
#     if adaptive_yaxis:
# #         ax.set_ylim(min(yaxis[i]), max(yaxis[i]))
#         try:
#             ax.set_ylim(yaxis[i][find_peaks(-yaxis[i])[0][0]], max(yaxis[i]))
#         except:
#             ax.set_ylim(min(yaxis[i]), max(yaxis[i]))
#     line.set_ydata(yaxis[i])  # update the data
#     return line,

# #Init only required for blitting to give a clean slate.
# def init():
#     line.set_ydata(np.ma.array(xaxis, mask=True))
#     return line,

# ani = animation.FuncAnimation(fig, animate, np.arange(1, int(len(t_exp)/down)-1), init_func=init,
#     interval=25, blit=True)
# HTML(ani.to_jshtml())



[100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113.
 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126. 127.
 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140. 141.
 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153. 154. 155.
 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167. 168. 169.
 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180. 181. 182. 183.
 184. 185. 186. 187. 188. 189. 190. 191. 192. 193. 194. 195. 196. 197.
 198. 199. 200. 201. 202. 203. 204. 205. 206. 207. 208. 209. 210. 211.
 212. 213. 214. 215. 216. 217. 218. 219. 220. 221. 222. 223. 224. 225.
 226. 227. 228. 229. 230. 231. 232. 233. 234. 235. 236. 237. 238. 239.
 240. 241. 242. 243. 244. 245. 246. 247. 248. 249. 250. 251. 252. 253.
 254. 255. 256. 257. 258. 259. 260. 261. 262. 263. 264. 265. 266. 267.
 268. 269. 270. 271. 272. 273. 274. 275. 276. 277. 278. 279. 280. 281.
 282. 283. 284. 285. 286. 287. 288. 289. 290. 291. 292. 293. 294. 295.
 296. 

In [33]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

X = y_points[::1]
Y = t_exp*1e6
X, Y = np.meshgrid(X, Y)
Z = y_pot[:,::1]

fig = plt.figure('3d')
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False, rstride=3, cstride=3)
ax.set_xlabel('\n position (um)', linespacing=3)
ax.set_ylabel('\n time (us)', linespacing=3)
ax.set_zlabel('\n potential (V)', linespacing=3)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [ ]:
#### Forward ITVG
# q_firstguess = q_min # initial guess for transport
q_firstguess = np.full(len(t_exp), q0) # initial guess for splitting
num_guess = 1

q = []
coeff = []
for n in range(num_guess):
    print('\t'.join(('iter', 't [us]', 'R [um]', 'f [Hz]', 'shift [um]')))
    if not n:
        qg = q_firstguess
    else:
        qg = q_guess
        
    expansion_point = [[[0., q, 51.]] for q in qg]
    
    for i in range(len(t_exp)):
        coeffs = volt_gen.compute_total_potential_axes(expansion_point[i], el_config[i], printing=False)
        coeff.append(coeffs)
        shift = -coeffs[1]/(2*coeffs[6])
        q.append(qg[i] + shift)
        print('\t'.join((str(i), '{:.2f}\t{:.2f}\t{:.0f}\t{:.3f}'.format(t_exp[i]*1e6, qg[i], get_freq_from_alpha(coeffs[6], Sr), shift))))
    q_guess = q[n*len(t_exp):(n+1)*len(t_exp)]

coeff_final = np.array(coeff[len(t_exp)*(num_guess-1):len(t_exp)*num_guess])
q_exp = np.array(q[len(t_exp)*(num_guess-1):len(t_exp)*num_guess])
a_exp = coeff_final.T[6]

In [ ]:
const_map = volt_gen.function_map
x_exp = coeff_final.T[const_map['x']]
y_exp = coeff_final.T[const_map['y']]
z_exp = coeff_final.T[const_map['z']]
c_exp = coeff_final.T[const_map['yyy']]
d_exp = coeff_final.T[const_map['yyyy']]
const_exp = coeff_final.T[const_map['const']]
# plt.plot(t_exp, x_exp)
# plt.plot(t_exp, y_exp)
# plt.plot(t_exp, z_exp)

# plt.plot(t_exp, q_exp)
# q_exp = np.repeat(q0, len(a_exp))


# Simulate Ions

**to do:**
- clean this bad boi up!!
- add 'lattice-type' simulation

In [34]:
def heat_sim(tff, q_protocol, t_eval, ms=1E-8, showplot=False, a=None, b=None, g=None, c=None, deg=2):
    ''' simulates ion motion and returns heat gained
        parameters: final integration time, motion protocol, time window for heat calculation 
        all length units must be in um'''
    if a is not None:
        alpha = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), np.append(a*1e12, a[-1]*1e12), k=deg)
    else:
        alpha = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), [get_alpha_from_freq(omega_0/(2*pi))]*(len(t_exp)+1), k=deg)
    if b is not None:
        beta = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), np.append(b*1e24, b[-1]*1e24), k=deg)
    else:
        beta = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), [0]*(len(t_exp)+1), k=deg)
    if g is not None:
        gamma = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), np.append(g*1e6, g[-1]*1e6), k=deg)
    else:
        gamma = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), [0]*(len(t_exp)+1), k=deg)
    if c is not None:
        cubic = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), np.append(c*1e18, c[-1]*1e18), k=deg)
    else:
        cubic = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), [0]*(len(t_exp)+1), k=deg)
    qi = InterpolatedUnivariateSpline(np.concatenate((t_exp,[tff*1.1])), np.concatenate((q_protocol/1e6, [q_protocol[-1]/1e6])), k=deg)
    
    def fn(t, v):
        x = v[:Nion]
        dxdt = v[Nion:]
        dist = get_dist_matrix(x)
        
        eom = [] ## equation of motion
        for i in range(Nion): ## iterate over all the ions
            eom.append(-q_ions[i]/m[i]*p_derivative(phi, point=[t, x[i]-qi.__call__(t), alpha.__call__(t), beta.__call__(t), m[i], gamma.__call__(t), cubic.__call__(t)]) +
                       k0*q_ions[i]/m[i]*sum([q_ions[j]*abs(dist[i][j])/(dist[i][j]**3) for j in range(Nion) if j!=i]))
        
        return np.concatenate([dxdt, eom])
        
    outsol = solve_ivp(fn, (0, tff), np.concatenate([x0, x0dot]), method='RK45', dense_output=True, t_eval=t_eval, max_step=ms)    
    h = get_heat(outsol.y[0]-qi.__call__(t_eval), 2*pi*get_freq_from_alpha(alpha.__call__(tff)/1e12, m[0]), m[0])
    
    if showplot:
        t_view = np.linspace(0, tff, int(1e9*tff))
        q = qi.__call__(t_view)
        x = outsol.sol(t_view)[0]

        fig, ax = plt.subplots()
        plt1, = ax.plot(t_view*1e6, q*1e6, 'k--')
#         ax.plot(t_view*1e6, x*1e6)
        ax.set_ylabel('potential minimum position (um)', color='k')
        ax2=ax.twinx()
        plt2, = ax2.plot(t_view*1E6, (x-q)*1E9)
#         plt2, = ax2.plot([], [])
        ax2.set_ylabel('ion oscillation (nm)')
        plt3 = ax2.axvline(t_eval[0]*1E6, color='r', linestyle='--')
        ax2.axvline(t_eval[-1]*1E6, color='r', linestyle='--')
        ax.set_xlabel('time (us)')
        plt.legend((plt1, plt2, plt3), ('potential minimum', 'ion motion', 'heat calculation window'), loc='best')
        
        for n in range(1, Nion):
            x = outsol.sol(t_view)[n]
            ax2.plot(t_view*1E6, (x-q)*1E9)
            
    return outsol, h, outsol.y, qi.__call__(t_eval)

def phi(t, y, a, b, m, g=0, c=0):
    ''' electric potential function [SI Units]
    y is the difference between ion position and potential minimum 
    '''
    return a*y**2 + b*y**4 + g*y + c*y**3

def p_derivative(func, ms=1e-11, var=1, point=[], order=1):
    ''' implements a partial derivative '''
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return derivative(wraps, point[var], dx = ms, n=order)
    
def get_dist_matrix(y):
    ''' gets distance matrix given positions of N ions 
        x = Nion x 1 vector '''
    D=np.zeros((Nion, Nion))
    for i in range(Nion):
        for j in range(i+1, Nion):
            D[i][j]=y[i]-y[j]
            D[j][i]=-D[i][j]
    return D

def get_heat(yvalues, w, m_=40*ct.m_u): ## calculates <n0> (average motional state) Given by Hercul 3.2 eq 97
    maxy = np.max(yvalues)
    miny = np.min(yvalues)
    A = (maxy-miny)/2
    return A**2*m_*np.abs(w)/(2*hbar)

def find_ic(q0, a0, N, xguess, b0=0, disp=False):
    ''' given q, phi coefficients, and num ion, minimize the total energy of the system
    both q and coeff have length scale in um'''
    def total_potential_energy(x, *args):
        q0, a0, b0 = args
        dist = get_dist_matrix(x)
        U = sum([a0*q_ions[i]*(x[i]-q0)**2 + b0*q_ions[i]*(x[i]-q0)**4 + 0.5*k0*q_ions[i]*sum([q_ions[j]/abs(dist[i][j])
                                                                   for j in range(N) if j!=i]) for i in range(N)])
        return U/1.6E-19
    idealx = minimize(total_potential_energy, xguess, args=(q0, a0, b0), method='SLSQP',
                     options={'ftol': 1e-12, 'eps': 1E-12, 'maxiter':1000})
    if disp:
        print(idealx)
    return idealx.x

In [41]:
hbar = ct.hbar
eps0 = ct.epsilon_0
k0 = 1/(4*pi*eps0)
amu = ct.m_u
elem_charge = ct.e
um = 1e-6

# t_exp = np.linspace(0, 5e-6, 100)
q_exp = np.full(len(t_exp), 186.5)
a_exp = np.full(len(t_exp), get_alpha_from_freq(1e6, m_=86*amu)/1e12)

Nion = 1
q_ions = elem_charge*   np.full(Nion, 1)
m = amu*                np.array([40])
x0dot = um*             np.full(Nion, 0)
x0guess = um*           np.array([q_exp[0]])

x0 = find_ic(q_exp[0]/1e6, a_exp[0]*1e12, Nion, x0guess)
# x0 = x0 * np.array([0.995, 1.001, 1.002])
print(x0*1e6)

[186.49999995]


In [ ]:
heat_start = 5e-6
heat_end = 55e-6

t_eval = np.linspace(heat_start, heat_end, 4096)
outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_exp, t_eval, showplot=True, a=a_exp, b=None, g=None, c=None, deg=1, ms=1e-8)
print('heat gained (quanta): ', n)

In [45]:
def heat_sim2(tff, t_eval, ypot_final, ms=1e-8, showplot=False, degx=1, degy=3):
    ''' simulates ion motion and returns heat gained
        parameters: final integration time, motion protocol, time window for heat calculation 
        all length units must be in um'''
    t_sim = np.concatenate((t_exp, [tff*1.1]))
    pot_sim = np.concatenate((ypot_final, ypot_final[-1:, :]), axis=0)
    y_sim = np.concatenate(([-1000e-6], y_points*1e-6, [1000e-6]))
    pot_sim = np.concatenate((pot_sim[:,:1], pot_sim, pot_sim[:,-1:]), axis=1)
    
    pot_func = RectBivariateSpline(t_sim, y_sim, pot_sim, kx=degx, ky=degy)
    
    def fn(t, v):
        x = v[:Nion]
        dxdt = v[Nion:]
        dist = get_dist_matrix(x)
        
        eom = [] ## equation of motion
        for i in range(Nion): ## iterate over all the ions
            eom.append(-q_ions[i]/m[i]*p_derivative(phi2, point=[t, x[i], pot_func]) +
                       k0*q_ions[i]/m[i]*sum([q_ions[j]*abs(dist[i][j])/(dist[i][j]**3) for j in range(Nion) if j!=i]))
        
        return np.concatenate([dxdt, eom])
        
    outsol = solve_ivp(fn, (0, tff), np.concatenate([x0, x0dot]), method='RK45', dense_output=True, t_eval=t_eval, max_step=ms)    
    
    if showplot:
        t_view = np.linspace(0, tff, int(1e9*tff))
        x = outsol.sol(t_view)[0]
        plt.plot(t_view*1e6, x*1e6)
#         plt.plot(t_view*1e6, outsol.sol(t_view)[1]*1e6)
#         plt.plot(t_view*1e6, outsol.sol(t_view)[2]*1e6)
#         plt.plot(t_view*1e6, outsol.sol(t_view)[3]*1e6)
#         plt.ylabel('ion position (um)', color='k')
        
    return outsol, outsol.y, pot_func

def phi2(t, y, pot_func):
    ''' electric potential function [SI Units]
    y is the difference global position 
    '''
    return float(pot_func.__call__(t, y))

def find_ic2(ypot, N, xguess, disp=False):
    ''' given q, phi coefficients, and num ion, minimize the total energy of the system
    both q and coeff have length scale in m'''
    def total_potential_energy(x, *args):
        ypot_func = args[0]
        dist = get_dist_matrix(x)
        U = sum([q_ions[i]*ypot_func.__call__(x[i]) + 0.5*k0*q_ions[i]*sum([q_ions[j]/abs(dist[i][j])
                                                                   for j in range(N) if j!=i]) for i in range(N)])
        return U/1.6e-19
    idealx = minimize(total_potential_energy, xguess, args=(ypot), method='Nelder-Mead', options={'xatol': 1e-9})
    if disp:
        print(idealx)
    return idealx.x

In [46]:
heat_start = 35e-6
heat_end = 45e-6

t_eval = np.linspace(heat_start, heat_end, 4096)
outsol, ion_eval, pot_grid = heat_sim2(t_eval[-1], t_eval, y_pot, showplot=True, ms=1e-8)
# endfreq = get_freq_from_alpha(a_exp[-1], Ca)
endfreq = f0
print(endfreq, 'Hz')
n = get_heat(ion_eval[0], 2*pi*endfreq, Ca)
print('heat gained (quanta): ', n)

4000000.0 Hz
heat gained (quanta):  0.3236721889082205
